In [209]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

## 定义一个简单的图

In [210]:
#边，shape = [2,num_edge]
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
#点，shape = [num_nodes, num_node_features]
x_node = torch.tensor([[0], [1], [2]], dtype=torch.float)

nodes_num = 3

### addSelfConnect函数：添加自环edge_index

In [211]:
selfconn = torch.stack([torch.range(0, nodes_num-1, dtype=torch.long)]*2, dim=0).to(edge_index.device) # 生成一个自环edge_index
print('step 1: 自环边矩阵') 
print(selfconn)

print('step 2: 合并自环边矩阵到原来的边矩阵上') 
print(torch.cat(tensors=[edge_index, selfconn], dim=1)) # 将自连接edge_index拼接到原本的edge_index中

def addSelfConnect(edge_index, nodes_num):
    selfconn = torch.stack([torch.range(0, nodes_num-1, dtype=torch.long)]*2, dim=0).to(edge_index.device)
    return torch.cat(tensors=[edge_index, selfconn], dim=1)

step 1: 自环边矩阵
tensor([[0, 1, 2],
        [0, 1, 2]])
step 2: 合并自环边矩阵到原来的边矩阵上
tensor([[0, 1, 1, 2, 0, 1, 2],
        [1, 0, 2, 1, 0, 1, 2]])
<ipython-input-211-8c5fa8eb251b>:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  selfconn = torch.stack([torch.range(0, nodes_num-1, dtype=torch.long)]*2, dim=0).to(edge_index.device) # 生成一个自环edge_index


In [224]:
edge_index = addSelfConnect(edge_index, x_node.shape[0])
edge_index

x = x_node

<ipython-input-211-8c5fa8eb251b>:9: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  selfconn = torch.stack([torch.range(0, nodes_num-1, dtype=torch.long)]*2, dim=0).to(edge_index.device)


## 定义一层线性变换

In [225]:
in_channel = 1
out_channel = 1
linear = nn.Linear(in_channel, out_channel)

In [226]:
print('step : 每个x都乘了个w（输出维度不一定是1）') 

# x = linear(x_node)
x

step : 每个x都乘了个w（输出维度不一定是1）


tensor([[0.],
        [1.],
        [2.]])

## 归一化信息

In [227]:
# 将edge_index矩阵拆分成两行
sour_edge, tar_edge = edge_index

print(sour_edge)
print(tar_edge)

tensor([0, 1, 1, 2, 0, 1, 2, 0, 1, 2])
tensor([1, 0, 2, 1, 0, 1, 2, 0, 1, 2])


### calDegree函数：计算节点的度

In [228]:
# nodes_num
# edges = sour_edge

# # 计算每个节点的度
# ind, deg = np.unique(edges.numpy(), return_counts=True)
# print('step 1:')
# print(ind)
# print(deg)

# # 生成每个节点的度tensor
# deg_tensor = torch.zeros((nodes_num, ), dtype=torch.long)
# print('step 2:')
# print(deg_tensor)

# deg_tensor[ind] = torch.from_numpy(deg)
# print('step 3:')
# print(deg_tensor)

# def calDegree(edges, nodes_num):
#         ind, deg = np.unique(edges.numpy(), return_counts=True)
#         deg_tensor = torch.zeros((nodes_num, ), dtype=torch.long)
#         deg_tensor[ind] = torch.from_numpy(deg)
#         return deg_tensor

# # def calDegree(edges, num_nodes):
# #         ind, deg = np.unique(edges.numpy(), return_counts=True)
# #         deg_tensor = torch.zeros((num_nodes ), dtype=torch.long)
# #         deg_tensor[ind] = torch.from_numpy(deg)
# #         return deg_tensor.to(edges.device)

$d^{-\frac{1}{2}}$

In [229]:
# deg = calDegree(edges, nodes_num).float()

# # 对度取根号
# deg_sqrt = deg.pow(-0.5) 
# print('step 1:')
# print(deg_sqrt)

# # 计算归一化系数
# norm = deg_sqrt[sour_edge] * deg_sqrt[tar_edge]
# print('step 2:')
# print(deg_sqrt[sour_edge])
# print(deg_sqrt[tar_edge])
# print(norm)

In [230]:
x

tensor([[0.],
        [1.],
        [2.]])

In [231]:
# deg.pow(-0.5) 

### 找出target_node的特征矩阵并进行归一化计算

In [232]:
# 找出target_node的特征矩阵
tar_matrix = torch.index_select(x, dim=0, index=tar_edge)
print('step 1:')
print(x)
print(tar_matrix)

# 归一化计算
# tar_matrix = norm.view(-1, 1) * tar_matrix
# print('step 2:')
# print(tar_matrix)

step 1:
tensor([[0.],
        [1.],
        [2.]])
tensor([[1.],
        [0.],
        [2.],
        [1.],
        [0.],
        [1.],
        [2.],
        [0.],
        [1.],
        [2.]])


## 聚合运算

In [233]:
V = nodes_num
H = tar_matrix

print(H)

# 建立n行source边矩阵，n为节点个数
mask = torch.stack([sour_edge] * V, 0)
print('step 1:')
print(mask)

# n行的source_edge分别减去0, 1, 2, ..., n
mask = mask.float() - torch.unsqueeze(torch.range(0, V-1).float(), 1) # 有点类似one-hot版的邻接矩阵
print('step 2:')
print(mask)

# 找出source边矩阵为零的元素，生成新的矩阵
mask = (mask == 0).float()
print('step 3:')
print(mask)

# 通过矩阵相乘进行aggregateSum
print('step 4:')
print(torch.mm(mask, H))

class AggrSum(nn.Module):
    def __init__(self, nodes_num):
        super(AggrSum, self).__init__()
        self.V = nodes_num
    
    def forward(self, H, sour_edge):
        # H : (E, c)
        # sour_edge : (E, 1)
        mask = torch.stack([sour_edge] * self.V, 0)
        mask = mask.float() - torch.unsqueeze(torch.range(0,self.V-1).float(), 1)
        mask = (mask == 0).float()
        # (N, E) * (E, c) -> (N, c)
        return torch.mm(mask, H)

tensor([[1.],
        [0.],
        [2.],
        [1.],
        [0.],
        [1.],
        [2.],
        [0.],
        [1.],
        [2.]])
step 1:
tensor([[0, 1, 1, 2, 0, 1, 2, 0, 1, 2],
        [0, 1, 1, 2, 0, 1, 2, 0, 1, 2],
        [0, 1, 1, 2, 0, 1, 2, 0, 1, 2]])
step 2:
tensor([[ 0.,  1.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.],
        [-1.,  0.,  0.,  1., -1.,  0.,  1., -1.,  0.,  1.],
        [-2., -1., -1.,  0., -2., -1.,  0., -2., -1.,  0.]])
step 3:
tensor([[1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 1., 1., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 1., 0., 0., 1.]])
step 4:
tensor([[1.],
        [4.],
        [5.]])
<ipython-input-233-26434730e2e1>:12: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  mask = mask.float() - torch.unsqueeze(torch.range(0, V-1).float(), 1) # 有点类似one-hot版的邻接矩阵


In [234]:
aggregation = AggrSum(nodes_num)
aggr = aggregation(tar_matrix, tar_edge)
H

<ipython-input-233-26434730e2e1>:34: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  mask = mask.float() - torch.unsqueeze(torch.range(0,self.V-1).float(), 1)


tensor([[1.],
        [0.],
        [2.],
        [1.],
        [0.],
        [1.],
        [2.],
        [0.],
        [1.],
        [2.]])

In [223]:
nodes_num

3